In [2]:
#################################################################################################
# THIS IS THE ONE THAT WORKS DO NOT DELETE
# sets up a keysight frequency counter to trigger on external voltage change
# collects data from the onboard buffer every few seconds saving to CSV file
#################################################################################################

import nidaqmx
import time
import pyvisa as visa
import pandas as pd
import Utilities as util
from os import path
import daqUtils
from pymeasure.instruments.keithley import Keithley2000
from pymeasure.adapters import PrologixAdapter

############ Helper functions go here ####################
def getTrigCountCmd(num):
    trig_num_cmd = 'TRIG:COUN ' + str(num)
    return trig_num_cmd

def getTrigSourceCmd(sr):
    trig_src_cmd = 'TRIG:SOUR '+ sr
    return trig_src_cmd
##########################################################

#do all the document prep at the beginning so it doesn't slow down collection later
#create a new csv file at the specified location
folder = 'Data\\WallTimeExperiment\\'
filename = util.dtStringForFilename()
fp = folder + filename
with open(fp, 'a') as creating_new_csv_file:
    pass
#create an empty data frame and save the headers to the file
df_headers = pd.DataFrame({'Dates':[], 'Frequencies': [], 'Voltages': [], 'Times':[], 'Timestamps':[]})
df_headers.to_csv(fp, mode='a', index=False)
print('Output file created')


##################################################################
# Set the your collection variables here
#
# trig_count = the number of measurements collected before that data is offloaded to a csv file
# trig_count_cmd = a SCPI command created from the specified trigger count
# trig_source = the trigger source for the instruments. It can be IMM, EXT, BUS, etc. For this it should be EXT
# trig_source_cmd = a SCPI command created from the specified trigger source
# high_V = the high voltage value to send as a trigger from the DAQ
# low_V = the low voltage value to send as a trigger from the DAQ, this should be 0.0
# high_time = the amount of time in seconds to send the high voltage 
# low_time = the amount of time in seconds to send the low voltage 
##################################################################
trig_count = 5
trig_count_cmd = getTrigCountCmd(trig_count)
trig_source = 'EXT'
trig_source_cmd = getTrigSourceCmd(trig_source)
high_V = 2.5
low_V = 0.0
high_time = 0.5
low_time = 0.5

# open the resource manager so we can connect to the keysight and the keithley
rm = visa.ResourceManager()

# Keysight connection setup
keysight_addr = 'USB0::0x0957::0x1807::MY50000728::INSTR'
freq_counter = rm.open_resource(keysight_addr)
freq_counter.encoding = 'latin_1'
freq_counter.source_channel = 'CH1'

# Keysight data collection set up
## reset everything and clear the event queues
freq_counter.write('*RST')
freq_counter.write('STAT:PRES')
freq_counter.write('*CLS')
## set the type of measurement to frequency
freq_counter.write('CONF:FREQ')
freq_counter.write(trig_source_cmd)
freq_counter.write('TRIG:SLOP POS')
freq_counter.write(trig_count_cmd)

# DAQ Setup and task initialization
task = nidaqmx.Task()
task.ao_channels.add_ao_voltage_chan("Dev1/ao0")
print('Starting Collection')
task.start()
task.write(0.0)#make sure we are starting at 0V

# Initialize the keysight
freq_counter.write('INIT')

#do one trigger cycle to get rid of the empty data point that apparently gets collected for reasons?
#I hate that this is a thing, but it appears to be a thing, so we're going to roll with it
task.write(2.0)
time.sleep(0.1)
task.write(0.0)
time.sleep(0.1)
freq_counter.query('R?')#remove the empty data point from the data register, so our time stamps will match up with the frequencies collected

# Continuing the things I hate, I have to wait until after the garbage cycle completes on the keysight to set up the dmm
# because if I don't it gets into a mystery state where it doesn't understand how to send data properly and I don't know why
# I would like to fix this correctly by figuring out how to reliably reset the dmm to the state it is in at power up, but since I 
# have yet to figure that out, we're doing this stupid work around
#Keithley dmm connection set up
Addr = 'ASRL1::INSTR' #address of USB connected to computer
adapter = PrologixAdapter('ASRL4::INSTR', 1) #create prologix adapter and connect to GPIB w/ address 1
dmm = Keithley2000(adapter) #create the instrument using the adapter

# Keithley data collection set up
## reset everything and clear the event queues
dmm.reset()
## need to set trigger type to external
dmm.write(trig_source_cmd)
## set trigger count to the desired number of datapoints per collection cycle
dmm.write(trig_count_cmd)
## set sample count to 1 (this is one sample per trigger)
dmm.write('SAMP:COUN 1')
# set up the data trace so I can get more than one mesurement when we reach the end of the collection cycle
# I have to do this here so all the data points stay in sync because of the weird empty value the keysight likes to for it's first trigger cycle
cmd_trace = 'TRAC:POIN ' + str(trig_count)
dmm.write(cmd_trace)
dmm.write('TRAC:FEED SENS1;FEED:CONT NEXT')
dmm.write('INIT')

how_many_cycles = 2
cycle_num=1

while (cycle_num<=how_many_cycles):
    print('Starting Data Collection Cycle '+str(cycle_num))
    times = [] #the timestamps will go here
    frequencies = ''
    dmm_vals = ''
    
    times = daqUtils.genAnalogTriggerCycle(task, trig_count, high_V, low_V, high_time, low_time)

    #task.write(0.0)# always end at 0.0V
    task.stop()
    print('Ending Collection Cycle '+str(cycle_num))

    try:
        frequencies = freq_counter.query('FETC?')
    except: 
        print('I regret to inform you that something done f*cked up with the Keysight and there is no data.')
        print('Ending data collection.')
        freq_counter.close()
        rm.close()
        break

    dmm_vals = dmm.ask('TRAC:DATA?')
        


    #now I need to make the data into some sort of format that we can easily put in a text file
    frequencies = util.stringToPandasSeries(frequencies, ',')
    dmm_vals = util.stringToPandasSeries(dmm_vals, ',')
    hrdates, hrtimes = util.formatTimestampsForCSV(times)
    
    df = pd.DataFrame({
        'Date': hrdates,
        'Frequency': frequencies,
        'Voltage': dmm_vals,
        'Time': hrtimes, 
        'Timestamps': times
    })
    df.to_csv(fp, mode='a', index=False, header=False)
    cycle_num = cycle_num+1


#close the connection
openres = rm.list_opened_resources()
print('Closing Connection with ', openres)
freq_counter.close()
rm.close()




Output file created
Starting Collection


c:\Users\saam\anaconda3\envs\ertw-1\lib\site-packages\pymeasure\instruments\generic_types.py:110: FutureWarning: It is not known whether this device support SCPI commands or not. Please inform the pymeasure maintainers if you know the answer.
  warn("It is not known whether this device support SCPI commands or not. Please inform "


Starting Data Collection Cycle 1
Ending Collection Cycle 1
Starting Data Collection Cycle 2
Ending Collection Cycle 2
Closing Connection with  [<'USBInstrument'('USB0::0x0957::0x1807::MY50000728::0::INSTR')>, <'SerialInstrument'('ASRL4::INSTR')>]


In [1]:
#################################################################################################
# THIS IS THE ONE THAT WORKS DO NOT DELETE
# sets up a keysight frequency counter to trigger on external voltage change
# collects data from the onboard buffer every few seconds saving to CSV file
#################################################################################################

import nidaqmx
import time
import pyvisa as visa
import pandas as pd
import Utilities as util
from os import path
import daqUtils
from pymeasure.instruments.keithley import Keithley2000
from pymeasure.adapters import PrologixAdapter

############ Helper functions go here ####################
def getTrigCountCmd(num):
    trig_num_cmd = 'TRIG:COUN ' + str(num)
    return trig_num_cmd

def getTrigSourceCmd(sr):
    trig_src_cmd = 'TRIG:SOUR '+ sr
    return trig_src_cmd
##########################################################

#do all the document prep at the beginning so it doesn't slow down collection later
#create a new csv file at the specified location
folder = 'Data\\WallTimeExperiment\\'
filename = util.dtStringForFilename()
fp = folder + filename
with open(fp, 'a') as creating_new_csv_file:
    pass
#create an empty data frame and save the headers to the file
df_headers = pd.DataFrame({'Dates':[], 'Frequencies': [], 'Voltages': [], 'Times':[], 'Timestamps':[]})
df_headers.to_csv(fp, mode='a', index=False)
print('Output file created')


##################################################################
# Set the your collection variables here
#
# trig_count = the number of measurements collected before that data is offloaded to a csv file
# trig_count_cmd = a SCPI command created from the specified trigger count
# trig_source = the trigger source for the instruments. It can be IMM, EXT, BUS, etc. For this it should be EXT
# trig_source_cmd = a SCPI command created from the specified trigger source
# high_V = the high voltage value to send as a trigger from the DAQ
# low_V = the low voltage value to send as a trigger from the DAQ, this should be 0.0
# high_time = the amount of time in seconds to send the high voltage 
# low_time = the amount of time in seconds to send the low voltage 
##################################################################
trig_count = 1
trig_count_cmd = getTrigCountCmd(trig_count)
trig_source = 'EXT'
trig_source_cmd = getTrigSourceCmd(trig_source)
high_V = 2.5
low_V = 0.0
high_time = 0.5
low_time = 0.5


# open the resource manager so we can connect to the keysight and the keithley
rm = visa.ResourceManager()

# Keysight connection setup
keysight_addr = 'USB0::0x0957::0x1807::MY50000728::INSTR'
freq_counter = rm.open_resource(keysight_addr)
freq_counter.encoding = 'latin_1'
freq_counter.source_channel = 'CH1'

# Keysight data collection set up
## reset everything and clear the event queues
freq_counter.write('*RST')
freq_counter.write('STAT:PRES')
freq_counter.write('*CLS')
## set the type of measurement to frequency
freq_counter.write('CONF:FREQ')
freq_counter.write(trig_source_cmd)
freq_counter.write('TRIG:SLOP POS')
freq_counter.write(trig_count_cmd)

#Keithley dmm connection set up
Addr = 'ASRL1::INSTR' #address of USB connected to computer
adapter = PrologixAdapter('ASRL4::INSTR', 1, gpib_read_timeout=1000) #create prologix adapter and connect to GPIB w/ address 1
dmm = Keithley2000(adapter) #create the instrument using the adapter

# Keithley data collection set up
## reset everything and clear the event queues
dmm.reset()
## need to set trigger type to external
dmm.write(trig_source_cmd)
## set trigger count to the desired number of datapoints per collection cycle
dmm.write(trig_count_cmd)
## set sample count to 1 (this is one sample per trigger)
dmm.write('SAMP:COUN 1')


# DAQ Setup and task initialization
task = nidaqmx.Task()
task.ao_channels.add_ao_voltage_chan("Dev1/ao0")
print('Starting Collection')
task.start()
task.write(0.0)#make sure we are starting at 0V

# Initialize the instruments
freq_counter.write('INIT')
dmm.write('INIT')

#do one trigger cycle to get rid of the empty data point that apparently gets collected for reasons?
#I hate that this is a thing, but it appears to be a thing, so we're going to roll with it
task.write(high_V)
time.sleep(0.1)
task.write(low_V)
time.sleep(0.1)
f = freq_counter.query('R?')#remove the empty data point from the data register, so our time stamps will match up with the frequencies collected
d = dmm.ask('FETC?')


print('Keysight: ' + str(f))
print('Keithley: ' + str(d))
    

#close the connection
openres = rm.list_opened_resources()
print('Closing Connection with ', openres)
freq_counter.close()
rm.close()




Output file created


c:\Users\saam\anaconda3\envs\ertw-1\lib\site-packages\pymeasure\instruments\generic_types.py:110: FutureWarning: It is not known whether this device support SCPI commands or not. Please inform the pymeasure maintainers if you know the answer.
  warn("It is not known whether this device support SCPI commands or not. Please inform "


Starting Collection
Keysight: #10

Keithley: -4.30537627E-01

Closing Connection with  [<'USBInstrument'('USB0::0x0957::0x1807::MY50000728::0::INSTR')>, <'SerialInstrument'('ASRL4::INSTR')>]


In [6]:
#################################################################################################
# Voltage testing module
# this module connects to the DAQ and creates an analog output task
# that task then sends 2V through the analog output for a few seconds before returning to 0V
# the purpose is to allow for testing of external triggers without having to hook up the actual instruments themselves
#################################################################################################

import nidaqmx
import time
import pyvisa as visa
import pandas as pd
import Utilities as util
from os import path
import daqUtils
import matplotlib as plt


task = nidaqmx.Task()
task.ao_channels.add_ao_voltage_chan("Dev1/ao0")
task.start()
task.write(0.0)#make sure we are starting at 0V
time.sleep(1)

#do one trigger cycle to get rid of the empty data point that apparently gets collected for reasons?
#I hate that this is a thing, but it appears to be a thing, so we're going to roll with it
task.write(2.0)
time.sleep(3)
task.write(0.0)
time.sleep(1)
task.stop()





In [1]:
import pyvisa as visa
from pymeasure.instruments.keithley import Keithley2000
from pymeasure.adapters import PrologixAdapter
import time
import nidaqmx

high_V = 4.0
low_V = 0.0
high_time = 0.5
low_time = 0.5


rm = visa.ResourceManager() #open this because I want to make sure I close all connections at the end

Addr = 'ASRL1::INSTR' #address of USB connected to computer
adapter = PrologixAdapter('ASRL4::INSTR', 1) #create prologix adapter and connect to GPIB w/ address 1
dmm = Keithley2000(adapter) #create the instrument using the adapter
dmm.reset()

'''
dmm.write('*RST')
dmm.write('STAT:PRES')
dmm.write('*CLS')
'''

#need to set trigger type to external
dmm.write('TRIG:SOUR EXT')
trig_source = dmm.ask('TRIG:SOUR?')
print(trig_source)

#set trigger count to something other than 1
dmm.write('TRIG:COUN 10')
trig_count = dmm.ask('TRIG:COUN?')
print(trig_count)

dmm.write('SAMP:COUN 1')

task = nidaqmx.Task()
task.ao_channels.add_ao_voltage_chan("Dev1/ao0")
print('Starting Collection')
task.start()
task.write(0.0)#make sure we are starting at 0

#read the voltage
cmd_trace = 'TRAC:POIN ' + str(trig_count)
dmm.write(cmd_trace)
dmm.write('TRAC:FEED SENS1;FEED:CONT NEXT')
dmm.write('INIT')

count = 0
while (count < int(trig_count)):
    task.write(high_V) #high voltage value to send (probably stay below 5V in general)
    time.sleep(high_time) #how long do we want to stay at the hight voltage
    task.write(low_V) #usually this will be 0V
    time.sleep(low_time) #how long do we want to stay at the low voltage
    count = count+1

task.write(0.0)
task.stop()
    

data = dmm.ask('TRAC:DATA?')
print(data)

rm.close()


c:\Users\saam\anaconda3\envs\ertw-1\lib\site-packages\pymeasure\instruments\generic_types.py:110: FutureWarning: It is not known whether this device support SCPI commands or not. Please inform the pymeasure maintainers if you know the answer.
  warn("It is not known whether this device support SCPI commands or not. Please inform "


EXT

10

Starting Collection


c:\Users\saam\anaconda3\envs\ertw-1\lib\site-packages\pymeasure\adapters\visa.py:156: UserWarning: write message already ends with termination characters
  self.connection.write(command, **kwargs)


-4.11536377E-01,-5.98526681E-01,-7.93246613E-01,-9.84018035E-01,-1.17318131E+00,-4.41012600E-01,-6.41538075E-01,-8.30210840E-01,-1.02251272E+00,-3.96807626E-01



In [ ]:
#################################################################################################
# COPY BECAUSE I AM PARANOID - THIS IS THE ONE THAT WORKS DO NOT DELETE
# sets up a keysight frequency counter to trigger on external voltage change
# collects data from the onboard buffer every few seconds saving to CSV file
#################################################################################################

import nidaqmx
import time
import pyvisa as visa
import pandas as pd
import Utilities as util
from os import path
import daqUtils
import matplotlib as plt

#do all the document prep at the beginning so it doesn't slow down collection later
#create a new csv file at the specified location
folder = 'Data\\WallTimeExperiment\\'
filename = util.dtStringForFilename()
fp = folder + filename
with open(fp, 'a') as creating_new_csv_file:
    pass
#create an empty data frame and save the headers to the file
df_headers = pd.DataFrame({'Dates':[], 'Frequencies': [], 'Times':[], 'Timestamps':[]})
df_headers.to_csv(fp, mode='a', index=False)
print('Output file created')

# open a connection with the keysight
rm = visa.ResourceManager()
# Input the target instrument IP address
Addr = 'USB0::0x0957::0x1807::MY50000728::INSTR'
inst = rm.open_resource(Addr)
inst.encoding = 'latin_1'
inst.source_channel = 'CH1'

trig_count = 5

# reset everything and clear the event queues
inst.write('*RST')
inst.write('STAT:PRES')
inst.write('*CLS')

# set the type of measurement to frequency
inst.write('CONF:FREQ')
inst.write('TRIG:SOUR EXT')
inst.write('TRIG:SLOP POS')
trignum_cmd = 'TRIG:COUNT '+str(trig_count)
inst.write(trignum_cmd)

task = nidaqmx.Task()
task.ao_channels.add_ao_voltage_chan("Dev1/ao0")
print('Starting Collection')
task.start()
task.write(0.0)#make sure we are starting at 0V
inst.write('INIT')

#do one trigger cycle to get rid of the empty data point that apparently gets collected for reasons?
#I hate that this is a thing, but it appears to be a thing, so we're going to roll with it
task.write(2.0)
time.sleep(0.1)
task.write(0.0)
time.sleep(0.1)
inst.query('R?')#remove the empty data point from the data register, so our time stamps will match up with the frequencies collected

how_many_cycles = 2
cycle_num=1

while (cycle_num<=how_many_cycles):
    print('Starting Data Collection Cycle '+str(cycle_num))
    times = [] #the timestamps will go here
    frequencies = ''
    
    times = daqUtils.genAnalogTriggerCycle(task, trig_count, 2.0, 0.0, 0.5, 0.5)

    #task.write(0.0)# always end at 0.0V
    task.stop()
    print('Ending Collection Cycle '+str(cycle_num))

    try:
        frequencies = inst.query('FETC?')
    except: 
        print('I regret to inform you that something done f*cked up and there is no data.')
        print('Ending data collection.')
        inst.close()
        rm.close()
        break


    #now I need to make the data into some sort of format that we can easily put in a text file
    frequencies = util.stringToPandasSeries(frequencies, ',')

    hrdates, hrtimes = util.formatTimestampsForCSV(times)
    
    df = pd.DataFrame({
        'Date': hrdates,
        'Frequency': frequencies,
        'Time': hrtimes, 
        'Timestamps': times
    })
    df.to_csv(fp, mode='a', index=False, header=False)
    cycle_num = cycle_num+1


#close the connection
openres = rm.list_opened_resources()
print('Closing Connection with ', openres)
inst.close()
rm.close()